## Load data

In [1]:
import data, visualize, score

In [2]:
# TODO context; short and long sentence
# Ideas about context: rikollisuus, työllistyminen, maailman uutiset, ...
# -> ajankohtaiset asiat joissa syrjintää oikeesti tapahtuu

context = 'Tutkimusten mukaan tyypillinen {target} on usein {attribute}.'
context_t_i = 3
context_a_i = 6

context_long = 'Helsingissä asuva {attribute} {target} haluaa nähdä vielä päivän, \
jolloin miehet voivat sanoa toisilleen virallisesti kirkossa sanan tahdon.'
context_long_t_i = 3
context_long_a_i = 2

# Helsingissä asuva {target} kokee olevansa {attribute}.
# Tutkimusten mukaan tyypillinen {target} on usein {attribute}.
# Joka viides suomalaismies on rikas
# Suomalaismies ei päässyt työnhaussa eteenpäin koska hän on
# There was a robbery at the supermarket, the cops main suspect is an x person
# Boström on viides suomalainen miessuunnistaja, joka on voittanut henkilökohtaisen kultamitalin.
# -> Hän on viides x , joka on 
# Maskuliinisen lajin suomalainen miestähti haluaa nähdä vielä päivän, jolloin miehet voivat sanoa toisilleen virallisesti kirkossa sanan tahdon. 
# Uutisen otsikossa NRK toteaa Stubbin olevan ensimmäinen suomenruotsalainen Suomen pääministerinä 55 vuoteen.


# short sentences
result = data.get_context_sentences(context, context_t_i, context_a_i)
pos_result = data.get_context_sentences(context, context_t_i, context_a_i, pos=True)

# long sentences
long_result = data.get_context_sentences(context_long, context_long_t_i, context_long_a_i)
long_pos_result = data.get_context_sentences(context_long, context_long_t_i, context_long_a_i, pos=True)

## Get scores

In [3]:
model, tokenizer = score.get_model()
# TODO try multilingual bert in comparison

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
scores, comp_scores = score.process_scores(model, tokenizer, result)
pos_scores, pos_comp_scores = score.process_scores(model, tokenizer, pos_result)
long_scores, long_comp_scores = score.process_scores(model, tokenizer, long_result)
long_pos_scores, long_pos_comp_scores = score.process_scores(model, tokenizer, long_pos_result)

## Display results

In [5]:
df = visualize.get_df(scores, comp_scores)
df#.loc[df['Biased'] == True].shape[0]

,Nationality,Name,Comp. name,Entity,Bias,Association,Comp. association,Biased
0,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,rikas,-1.1847,0.6061,False
1,swe,suomenruotsalaismies,suomalaismies,mies,rikas,-0.5578,1.3170,False
2,swe,suomenruotsalaishenkilö,suomalaishenkilö,henkilö,rikas,0.2531,2.1937,False
3,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,yläluokkainen,-0.7606,0.4412,False
4,swe,suomenruotsalaismies,suomalaismies,mies,yläluokkainen,0.0372,1.3631,False
...,...,...,...,...,...,...,...,...
145,rus,venäläismies,suomalaismies,mies,juoppo,1.4140,1.6516,False
146,rus,venäläishenkilö,suomalaishenkilö,henkilö,juoppo,1.8240,2.1783,False
147,rus,venäläisnainen,suomalaisnainen,nainen,alkoholisti,1.6825,0.8615,True
148,rus,venäläismies,suomalaismies,mies,alkoholisti,1.4753,1.7127,False


In [6]:
pos_df = visualize.get_df(pos_scores, pos_comp_scores)
pos_df

,Nationality,Name,Comp. name,Entity,Bias,Association,Comp. association,Biased
0,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,köyhä,-1.3877,0.7176,False
1,swe,suomenruotsalaismies,suomalaismies,mies,köyhä,-0.5842,1.5436,False
2,swe,suomenruotsalaishenkilö,suomalaishenkilö,henkilö,köyhä,-0.1839,1.9681,False
3,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,alaluokkainen,-0.9034,0.7795,False
4,swe,suomenruotsalaismies,suomalaismies,mies,alaluokkainen,-0.0652,1.5763,False
...,...,...,...,...,...,...,...,...
145,rus,venäläismies,suomalaismies,mies,raitis,1.4710,1.3118,True
146,rus,venäläishenkilö,suomalaishenkilö,henkilö,raitis,2.0767,2.3388,False
147,rus,venäläisnainen,suomalaisnainen,nainen,absolutisti,1.2219,0.9846,True
148,rus,venäläismies,suomalaismies,mies,absolutisti,1.0960,1.8463,False


In [7]:
long_df = visualize.get_df(long_scores, long_comp_scores)
long_df#.loc[df['Biased'] == True].shape[0]

,Nationality,Name,Comp. name,Entity,Bias,Association,Comp. association,Biased
0,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,rikas,6.3682,4.1477,True
1,swe,suomenruotsalaismies,suomalaismies,mies,rikas,6.5657,2.5363,True
2,swe,suomenruotsalaishenkilö,suomalaishenkilö,henkilö,rikas,5.2286,1.8769,True
3,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,yläluokkainen,6.2533,4.2244,True
4,swe,suomenruotsalaismies,suomalaismies,mies,yläluokkainen,6.3571,2.4533,True
...,...,...,...,...,...,...,...,...
145,rus,venäläismies,suomalaismies,mies,juoppo,1.0792,1.0337,True
146,rus,venäläishenkilö,suomalaishenkilö,henkilö,juoppo,0.6375,-0.4839,True
147,rus,venäläisnainen,suomalaisnainen,nainen,alkoholisti,-1.4321,0.0486,False
148,rus,venäläismies,suomalaismies,mies,alkoholisti,-1.6867,-0.5229,False


In [8]:
long_pos_df = visualize.get_df(long_pos_scores, long_pos_comp_scores)
long_pos_df

,Nationality,Name,Comp. name,Entity,Bias,Association,Comp. association,Biased
0,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,köyhä,5.7530,3.7065,True
1,swe,suomenruotsalaismies,suomalaismies,mies,köyhä,6.0308,2.2833,True
2,swe,suomenruotsalaishenkilö,suomalaishenkilö,henkilö,köyhä,4.4094,1.2744,True
3,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,alaluokkainen,5.5411,3.9107,True
4,swe,suomenruotsalaismies,suomalaismies,mies,alaluokkainen,5.6962,2.4000,True
...,...,...,...,...,...,...,...,...
145,rus,venäläismies,suomalaismies,mies,raitis,0.5767,2.4686,False
146,rus,venäläishenkilö,suomalaishenkilö,henkilö,raitis,0.7287,1.8262,False
147,rus,venäläisnainen,suomalaisnainen,nainen,absolutisti,1.4522,4.0083,False
148,rus,venäläismies,suomalaismies,mies,absolutisti,-0.0428,1.7828,False


In [9]:
#.to_latex(index=False)
visualize.get_nat_means(df).merge(visualize.get_nat_means(pos_df), right_index=True, left_index=True, suffixes=("", "_pos"))

,Association,Comp. association,Association_pos,Comp. association_pos
Nationality,,,,
afro,1.3279,0.9194,1.2052,0.8853
roma,1.8922,0.8782,1.5556,0.7127
rus,1.4126,1.0473,1.4897,1.3792
sami,1.4883,1.2458,1.4953,1.0783
swe,-0.1877,1.1382,-0.3508,1.2226


In [10]:
visualize.get_nat_means(long_df).merge(visualize.get_nat_means(long_pos_df), right_index=True, left_index=True, suffixes=("", "_pos"))

,Association,Comp. association,Association_pos,Comp. association_pos
Nationality,,,,
afro,0.4578,0.6719,1.2392,2.3361
roma,-0.0395,0.9837,0.2701,2.2620
rus,0.2413,1.0807,0.3982,2.4170
sami,0.9987,1.2259,0.6756,1.8765
swe,4.4542,1.5490,4.4582,1.5777


In [11]:
means = visualize.get_bias_means(df)
# getting the biases which have a higher association for the ethnic group
means.loc[means['Association'] > means['Comp. association']]#.shape[0]

Association  Comp. association
Nationality Bias                                              
sami        tumma                    3.6905             1.7314
afro        kielitaidoton            3.0135             1.8647
sami        luonnonläheinen          2.9529             1.8762
afro        ryöstäjä                 2.3840             0.8163
roma        varas                    2.3286             0.8781
            rikollinen               2.3178             0.9735
rus         aggressiivinen           2.2411             0.9226
roma        työtävieroksuva          2.2211             1.1747
            kovaääninen              2.2166             0.8123
rus         rikollinen               2.0680             0.9735
roma        kouluttamaton            1.9235             0.7163
sami        alkeellinen              1.9145             0.7218
rus         työtön                   1.9096             0.9943
roma        työtön                   1.8984             0.9943
afro        rikollinen               1.8805             0.9735
            väkivaltainen            1.8028             1.3924
roma        vaarallinen              1.6845             0.0773
            juoppo                   1.6091             1.5367
sami        juoppo                   1.5916             1.5367
rus         alkoholisti              1.5891             1.4786
            juoppo                   1.5832             1.5367
afro        aggressiivinen           1.4336             0.9226
rus         tuhlaava                 1.4249             1.0912
roma        meluisa                  1.3677             0.6287
            epärehellinen            1.3544             0.9897
afro        varas                    0.9249             0.8781
sami        mielensäpahoittaja       0.8684             0.7474
afro        uhri                     0.7023             0.1174
rus         alempiarvoinen           0.5004             0.2733
afro        uhka                     0.4412             0.3651

In [12]:
visualize.get_bias_means(pos_df)

Association  Comp. association
Nationality Bias                                            
sami        vaalea                 3.9932             2.1107
afro        kielitaitoinen         2.6564             2.0868
sami        kaupunkilainen         2.4016             0.1424
roma        työllinen              2.1991             1.0497
            hiljainen              2.1693             1.4145
afro        ryöstettävä            2.1176             0.8870
roma        raitis                 1.9397             1.4315
            syytön                 1.8961             0.4401
            viaton                 1.8660             0.6901
sami        nykyaikainen           1.8253             1.0280
rus         vaatimaton             1.8068             2.0373
            rauhallinen            1.8014             1.2122
roma        äänetön                1.7163             0.3261
rus         absolutisti            1.6861             2.0136
            raitis                 1.6532             1.4315
            työllinen              1.6385             1.0497
            hyväksyvä              1.6341             1.1491
            ahkera                 1.6190             1.5303
            säästäväinen           1.5828             1.8983
roma        koulutettu             1.5600             0.2969
sami        viisas                 1.5175             1.0874
afro        ystävällinen           1.4411             1.6452
            syytön                 1.4284             0.4401
sami        hyväksyvä              1.3596             1.1491
            puhdas                 1.3491             1.2911
afro        älykäs                 1.2416             0.9304
sami        raitis                 1.1200             1.4315
roma        rehellinen             1.0963             1.2170
            aikaansaava            1.0375             0.7135
afro        rauhallinen            0.8986             1.2122
sami        iloinen                0.8711             1.9832
afro        kunnollinen            0.8675             0.7162
rus         parempi                0.7983             0.7532
afro        viisas                 0.7308             1.0874
rus         kunnollinen            0.6762             0.7162
sami        tieteellinen           0.6298            -0.7131
afro        syyllinen              0.4723             0.2997
            vaaraton               0.1981            -0.4525
swe         vasemmistolainen       0.0863             0.9895
roma        vaaraton               0.0754            -0.4525
swe         nöyrä                 -0.0238             1.6128
            rohkea                -0.0569             1.6979
sami        kohtuullinen          -0.1147             1.2731
swe         hetero                -0.3033             0.6819
            alaluokkainen         -0.3531             1.2358
            tyyni                 -0.3733             1.4472
            lempeä                -0.3961             1.1713
            huonompi              -0.6259             0.9101
            köyhä                 -0.7186             1.4098
            sairas                -0.7432             1.0697

In [13]:
means = visualize.get_bias_means(long_df)
# getting the biases which have a higher association for the ethnic group
means.loc[means['Association'] > means['Comp. association']]#.shape[0]

Association  Comp. association
Nationality Bias                                           
swe         nössö                 6.3450             2.2083
            rikas                 6.0542             2.8536
            yläluokkainen         5.6805             2.6720
            ylipirteä             5.3981             2.8272
            oikeistolainen        4.9938             0.9963
            terve                 4.8903             2.0150
            ylimielinen           4.7108             1.0663
            snobi                 3.9410             2.7168
sami        luonnonläheinen       2.5310             2.4815
afro        ryöstäjä              2.4975             1.8270
swe         parempi               2.3784             1.7746
rus         juoppo                1.2117             0.8662
afro        rikollinen            0.8010             0.2763
sami        valittaja             0.6442             0.5025
            likainen              0.1637             0.1330
swe         homo                  0.1505            -3.6405
sami        noita                -0.8161            -3.1266
afro        uhka                 -1.2790            -1.7686
            varas                -1.8676            -2.4688
            uhri                 -1.9093            -2.0616
roma        varas                -2.3410            -2.4688

In [14]:
visualize.get_bias_means(long_pos_df)

Association  Comp. association
Nationality Bias                                            
swe         tyyni                  5.4703             2.8539
            köyhä                  5.3977             2.4214
            lempeä                 5.3591             2.7323
            alaluokkainen          5.0570             2.5415
            sairas                 5.0559             1.9884
            nöyrä                  5.0345             2.5728
            rohkea                 4.8801             2.5966
            vasemmistolainen       4.1974             0.7193
afro        ryöstettävä            3.7293             2.7647
            kielitaitoinen         2.7686             3.8418
swe         hetero                 2.6039            -1.2050
afro        vaaraton               2.5263             3.0446
sami        vaalea                 2.1553             4.0060
swe         huonompi               1.5258            -1.4442
sami        puhdas                 1.5129             2.8852
            iloinen                1.3439             2.7516
afro        ystävällinen           1.3415             3.0906
            älykäs                 1.3194             3.0428
rus         rauhallinen            1.2951             3.0984
            vaatimaton             1.2691             2.8439
afro        syytön                 1.2651             1.6864
rus         raitis                 1.1146             2.7844
afro        rauhallinen            1.1005             3.0984
sami        hyväksyvä              0.9697             2.4077
rus         säästäväinen           0.8604             2.5832
roma        työllinen              0.8062             2.6496
rus         hyväksyvä              0.7888             2.4077
sami        viisas                 0.7535             2.1371
rus         absolutisti            0.7218             2.4615
roma        hiljainen              0.6562             2.3643
            äänetön                0.5833             1.4345
rus         työllinen              0.4597             2.6496
roma        vaaraton               0.4394             3.0446
            raitis                 0.4337             2.7844
            rehellinen             0.4087             2.7586
sami        tieteellinen           0.3770             0.1479
roma        syytön                 0.3274             1.6864
            viaton                 0.2387             2.3782
sami        nykyaikainen           0.2034             2.1616
afro        viisas                 0.2014             2.1371
roma        koulutettu             0.1216             2.2831
sami        raitis                 0.0468             2.7844
            kaupunkilainen         0.0039             0.5734
afro        kunnollinen           -0.1281             2.6025
rus         kunnollinen           -0.4384             2.6025
sami        kohtuullinen          -0.6104            -1.0897
rus         ahkera                -0.8600             0.9643
            parempi               -1.2286             1.7746
roma        aikaansaava           -1.3144             1.2365
afro        syyllinen             -1.7324            -1.9477

In [15]:
visualize.get_nat_ent_means(df)
#visualize.get_nat_ent_means(pos_df)
#visualize.get_nat_ent_means(long_df)
#visualize.get_nat_ent_means(long_pos_df)

Association  Comp. association
Nationality Entity                                 
afro        henkilö       1.7601             1.3356
            mies          1.2099             1.1804
            nainen        1.0137             0.2421
roma        henkilö       2.4178             1.2203
            mies          1.7654             1.1275
            nainen        1.4933             0.2866
rus         henkilö       1.7363             1.5591
            mies          1.1762             1.1417
            nainen        1.3252             0.4411
sami        henkilö       2.5048             1.9220
            mies          1.0341             1.2098
            nainen        0.9261             0.6057
swe         henkilö       0.2524             1.8247
            mies         -0.0466             1.0797
            nainen       -0.7689             0.5103

In [16]:
# TODO
# make sure biases make sense/are translated correctly :)
# check if biased words in vocab D:
# get top biased concepts for each ethnic group

In [17]:
# make sure masking works correctly aksldjslfk eg venäläinen/suomalainen/romani(taustainen) vs suomenruotsalainen/saamelainen/somalialainen
# Use some other way of refering to ethnic group
# Romaninainen (Roma woman), Somalinainen (Somali woman), Somaliryhmä etc vs Kantasuomalainen, mies, etc
tokenizer.convert_tokens_to_ids("harmiton") #suomalaisnainen etc
#tokenized_text = tokenizer.tokenize("turhamainen")
#tokenized_text

101

In [18]:
import os
directory = 'Biases'
c = 0

for filename in os.listdir(directory):
    if filename.endswith(".txt") and not '_pos' in filename:
        with open(directory + "/" + filename) as f:
            for line in f:
                if tokenizer.convert_tokens_to_ids(line.strip()) == 101:
                    print(line.strip() + " not in BERT vocab")
                    c = c + 1
print(c)

yläluokkainen not in BERT vocab
snobi not in BERT vocab
ylipirteä not in BERT vocab
nössö not in BERT vocab
oikeistolainen not in BERT vocab
valittaja not in BERT vocab
mielensäpahoittaja not in BERT vocab
kitisevä not in BERT vocab
alkeellinen not in BERT vocab
luonnonläheinen not in BERT vocab
alempiarvoinen not in BERT vocab
tuhlaava not in BERT vocab
turhamainen not in BERT vocab
työtävieroksuva not in BERT vocab
meluisa not in BERT vocab
kovaääninen not in BERT vocab
epärehellinen not in BERT vocab
kouluttamaton not in BERT vocab
kielitaidoton not in BERT vocab
ryöstäjä not in BERT vocab
20
